In [5]:
import numpy as np
import pyamg
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LinearRegression
import os

import scipy

%matplotlib notebook

In [13]:
files = [f for f in os.listdir('.') if os.path.isfile(f) and '.mat' in f]

for f in files:
    mat_data = scipy.io.loadmat(f)
    matrix = mat_data[list(mat_data.keys())[-1]]
    
    

In [100]:
A = scipy.io.loadmat('BCSSTK16.mat')['A16']
E = np.ma.filled(np.ma.masked_array(np.random.normal(scale=1e-1, size=(A.shape[0],A.shape[1])), 
                                                mask=1 - 1 * (A != 0).todense()), fill_value=0)
Ap = A + E

In [88]:
b = np.random.rand(A.shape[0])

In [61]:
def cg(A, b):
    return pyamg.krylov.cg(A, b, tol=1e-10, maxiter=1000)[0]

def mg(A, b):
    return pyamg.ruge_stuben_solver(A).solve(b, tol=1e-10)

def sd(A, b):
    return pyamg.krylov.steepest_descent(A, b, tol=1e-10)[0]

def gmres(A, b):
    return pyamg.krylov.gmres(A, b, tol=1e-10, maxiter=1000)[0]


In [97]:
#mg_sol = mg(A, b)
#sd_sol = sd(A, b)
gm_sol = gmres(A, b)
#cg_sol = cg(A,b)

#mg_sol2 = mg(Ap, b)
#sd_sol2 = sd(Ap, b)
gm_sol2 = gmres(Ap, b)
#cg_sol2 = cg(Ap,b)

#sol_sol = np.linalg.solve(scipy.sparse.csr_matrix.toarray(A),b)

In [81]:
np.linalg.norm(cg_sol2 - cg_sol)/np.linalg.norm(cg_sol - sol_sol)

3.559865133657781e-17

In [93]:
np.linalg.norm(mg_sol2 - mg_sol) #1e-3

0.00485559487485187

In [90]:
np.linalg.norm(mg_sol2 - mg_sol) #1e-5

6.690607863153364e-05

In [82]:
np.linalg.norm(sd_sol2 - sd_sol)/np.linalg.norm(sd_sol - sol_sol)

2.0751352591082159e-16

In [95]:
np.linalg.norm(gm_sol2 - gm_sol) #1e-3

0.004854241164058914

In [98]:
np.linalg.norm(gm_sol2 - gm_sol) #1e-1

0.530684443062038